In [ ]:
!pip install --upgrade -q "transformers[torch]" datasets evaluate accelerate

In [ ]:
import pandas as pd
import ast
import re
import numpy as np

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import evaluate

from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import torch

In [ ]:
agg_df = pd.read_csv("/kaggle/input/financial-sentiment-aggregated/combined_data.csv")
print(len(agg_df))
df = agg_df[[
    'cleaned_text',
    'target'
]].rename(columns={'cleaned_text': 'text', 'target':'label'})
display(df.head(4))

In [ ]:
train_val_df,test_df = train_test_split(
    df,
    test_size = 0.15, # 15% for final evaluation
    stratify = df['label']
)
train_df,val_df = train_test_split(
    train_val_df,
    test_size = 0.1, # 10% for validation
    stratify = train_val_df['label']
)
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

# Convert into Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

ds = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(ds)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    text_to_tokenize = [str(text) for text in examples["text"]]
    # Pass the entire list directly to the tokenizer
    return tokenizer(text_to_tokenize, truncation=True, padding=True)
tokenized_ds = ds.map(preprocess_function, batched = True)

In [ ]:
# Define the label mappings
id2label = {0: "negative", 1: "positive", 2: "neutral"}
label2id = {"negative": 0, "positive": 1, "neutral": 2}

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

# Define our metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

# Get the token
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

training_args = TrainingArguments(
    output_dir="distilbert-financial-sentiment",
    report_to="tensorboard",  # see logs under "Advanced Settings" -> "TensorBoard"
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,

    eval_strategy="epoch", # Evaluate at the end of each epoch
    save_strategy="epoch", # Save a checkpoint at the end of each epoch
    
    load_best_model_at_end=True,  # Automatically load the best model when training is done
    push_to_hub=True, # Push the final model to your Hugging Face Hub profile
)

# This object encapsulates the entire training and evaluation process.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"], 
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Starting training with the modern Trainer API...")
trainer.train()
print("Training complete.")

trainer.push_to_hub()
print("Best model pushed to the Hub.")